<a href="https://colab.research.google.com/github/NoahTheChainer/Capaciatated-Plant-Location/blob/main/Capaciatated_Plant_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp
from pulp import *
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 80.7 MB/s eta 0:00:00


In [ ]:
# Demand table
demand = pd.DataFrame({
    'Supply_Region': ['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'Dmd': [2719.6, 84.1, 1676.8, 145.4, 156.4]
})
demand.set_index('Supply_Region', inplace=True)

# Variable cost table
var_cost = pd.DataFrame({
    'Supply_Region': ['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'USA': [6, 13, 20, 12, 17],
    'Germany': [13, 6, 14, 14, 13],
    'Japan': [20, 14, 3, 21, 9],
    'Brazil': [12, 14, 21, 8, 21],
    'India': [22, 13, 10, 23, 8]
})
var_cost.set_index('Supply_Region', inplace=True)

# Fix cost table
fix_cost = pd.DataFrame({
    'Supply_Region': ['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'Low_Cap': [6500, 4980, 6230, 3230, 2110],
    'High_Cap': [9500, 7270, 9100, 4730, 3080]
})
fix_cost.set_index('Supply_Region', inplace=True)

# Capabilities table
cap = pd.DataFrame({
    'Supply_Region': ['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'Low_Cap': [500, 500, 500, 500, 500],
    'High_Cap': [1500, 1500, 1500, 1500, 1500]
})
cap.set_index('Supply_Region', inplace=True)

In [ ]:
# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)

# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']
x = LpVariable.dicts("production_",
                     [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts("plant_",
                     [(i,s) for s in size for i in loc], cat='Binary')

/usr/local/lib/python3.11/dist-packages/pulp/pulp.py:1423: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
# Define objective function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)]
                 for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)]
                   for i in loc for j in loc]))

In [ ]:
# Define the constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Dmd']
for i in loc:
    model += lpSum([x[(i,j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[i,s]
                                                       for s in size])


In [ ]:
model.solve()

# Print the decision variables
print(LpStatus[model.status])

print("Objective = ", value(model.objective))

print("\nDecision Variables")
for v in model.variables():
    print(v.name, "=", v.varValue)

print("\nConstraints")
for name, constraint in model.constraints.items():
    print(name, ":", constraint.value())

Optimal
Objective =  58674.09999999999

Decision Variables
plant__('Brazil',_'High_Cap') = 1.0
plant__('Brazil',_'Low_Cap') = 0.0
plant__('Germany',_'High_Cap') = 0.0
plant__('Germany',_'Low_Cap') = 0.0
plant__('India',_'High_Cap') = 0.0
plant__('India',_'Low_Cap') = 1.0
plant__('Japan',_'High_Cap') = 1.0
plant__('Japan',_'Low_Cap') = 0.0
plant__('USA',_'High_Cap') = 1.0
plant__('USA',_'Low_Cap') = 0.0
production__('Brazil',_'Brazil') = 145.4
production__('Brazil',_'Germany') = 0.0
production__('Brazil',_'India') = 0.0
production__('Brazil',_'Japan') = 0.0
production__('Brazil',_'USA') = 1219.6
production__('Germany',_'Brazil') = 0.0
production__('Germany',_'Germany') = 0.0
production__('Germany',_'India') = 0.0
production__('Germany',_'Japan') = 0.0
production__('Germany',_'USA') = 0.0
production__('India',_'Brazil') = 0.0
production__('India',_'Germany') = 84.1
production__('India',_'India') = 156.4
production__('India',_'Japan') = 176.8
production__('India',_'USA') = 0.0
production_